# Análisis estático y dinámico

In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Análisis estático


In [21]:
import pefile
from datetime import datetime, timezone
from capstone import Cs, CS_ARCH_X86, CS_MODE_32

In [3]:


pe = pefile.PE("examples/sample_vg655_25th.exe")


print("Secciones del ejecutable:")
for section in pe.sections:
    print(f"Nombre: {section.Name.decode().strip()}, Tamaño Raw: {section.SizeOfRawData}, Tamaño Virtual: {section.Misc_VirtualSize}")

# Buscar posibles indicadores de empaquetado (UPX)
for section in pe.sections:
    if b"UPX" in section.Name:
        print("\n🔍 Posible empaquetado detectado con UPX.")
        break
else:
    print("\n✅ No se detectaron indicios claros de empaquetado.")

Secciones del ejecutable:
Nombre: .text   , Tamaño Raw: 28672, Tamaño Virtual: 27056
Nombre: .rdata  , Tamaño Raw: 24576, Tamaño Virtual: 24432
Nombre: .data   , Tamaño Raw: 8192, Tamaño Virtual: 6488
Nombre: .rsrc   , Tamaño Raw: 3448832, Tamaño Virtual: 3448736

✅ No se detectaron indicios claros de empaquetado.


1. 
    - ¿Se encuentran empaquetados los ejecutable?
        * No hay secciones con nombres como .UPX0 o .UPX1.
        * Los tamaños Raw y Virtual son consistentes (cuando están empaquetados, suele haber una gran diferencia)
    
    - ¿Cómo lo puede determinar?
        * Se verificaron las secciones del ejecutable y no se encontraron patrones relacionados con empaquetadores conocidos como UPX
        * No hay una diferencia muy notable entre el tamaño en crudo y el virtual
        * No hay nombres o terminacines con .UPX



In [4]:
suspicious_api_calls = {
    "process hollowing": {
        "CreateProcess", "GetModuleHandle", "GetProcAddress", "VirtualAllocEx", "WriteProcessMemory", "SetThreadContext", "ResumeThread"
    },
    "create remote thread": {
        "OpenProcess", "GetModuleHandle", "GetProcAddress", "VirtualAllocEx", "WriteProcessMemory", "CreateRemoteThread"
    },
    "enumerating processes": {
        "CreateToolhelp32Snapshot", "Process32First", "Process32Next", "WTSEnumerateProcesses"
    },
    "drop file from PE resource": {
        "GetModuleHandle", "FindResource", "LoadResource", "CreateFileA"
    },
    "IAT hooking": {
        "GetModuleHandle", "strcmp", "VirtualProtect"
    },
    "delete itself": {
        "GetModuleFileName", "ExitProcess", "DeleteFile"
    },
    "download and execute PE file": {
        "URLDownloadToFile", "ShellExecute"
    },
    "bind TCP port": {
        "WSAStartup", "socket"
    },
    "capture network traffic": {
        "socket", "bind", "WSAIoctl", "recvfrom"
    }
}

In [5]:
dll_function_usage = {}
suspicious_matches = {}

print("\n🔍 Análisis de DLLs y funciones utilizadas:")
for entry in getattr(pe, 'DIRECTORY_ENTRY_IMPORT', []):
    dll_name = entry.dll.decode('utf-8')
    dll_function_usage[dll_name] = []
    print(f"\n📁 DLL: {dll_name}")
    for imp in entry.imports:
        func_name = imp.name.decode('utf-8') if imp.name else "N/A"
        dll_function_usage[dll_name].append(func_name)
        print(f"    ↳ {func_name}")

        # Comparar con las llamadas maliciosas de la tabla 3
        for activity, apis in suspicious_api_calls.items():
            if func_name in apis:
                suspicious_matches.setdefault(activity, []).append(func_name)

# Imprimir resultados de comparación
if suspicious_matches:
    print("\nSe encontraron posibles comportamientos maliciosos:")
    for activity, functions in suspicious_matches.items():
        print(f"- {activity}: {', '.join(functions)}")
else:
    print("\nNo se encontraron comportamientos maliciosos según la Tabla 3.")

# Cerrar el archivo PE
pe.close()


🔍 Análisis de DLLs y funciones utilizadas:

📁 DLL: KERNEL32.dll
    ↳ GetFileAttributesW
    ↳ GetFileSizeEx
    ↳ CreateFileA
    ↳ InitializeCriticalSection
    ↳ DeleteCriticalSection
    ↳ ReadFile
    ↳ GetFileSize
    ↳ WriteFile
    ↳ LeaveCriticalSection
    ↳ EnterCriticalSection
    ↳ SetFileAttributesW
    ↳ SetCurrentDirectoryW
    ↳ CreateDirectoryW
    ↳ GetTempPathW
    ↳ GetWindowsDirectoryW
    ↳ GetFileAttributesA
    ↳ SizeofResource
    ↳ LockResource
    ↳ LoadResource
    ↳ MultiByteToWideChar
    ↳ Sleep
    ↳ OpenMutexA
    ↳ GetFullPathNameA
    ↳ CopyFileA
    ↳ GetModuleFileNameA
    ↳ VirtualAlloc
    ↳ VirtualFree
    ↳ FreeLibrary
    ↳ HeapAlloc
    ↳ GetProcessHeap
    ↳ GetModuleHandleA
    ↳ SetLastError
    ↳ VirtualProtect
    ↳ IsBadReadPtr
    ↳ HeapFree
    ↳ SystemTimeToFileTime
    ↳ LocalFileTimeToFileTime
    ↳ CreateDirectoryA
    ↳ GetStartupInfoA
    ↳ SetFilePointer
    ↳ SetFileTime
    ↳ GetComputerNameW
    ↳ GetCurrentDirectoryA
    ↳

2. 
    - ¿Hay indicios de algún comportamiento malicioso en base a la comparación? Justifique su respuesta.
        * Sí, si hay indicios. 
        * CreateFileA y LoadResource son funciones se usan a veces para sacar archivos escondidos de un programa, algo típico de malware que quiere dejar archivos maliciosos
        * VirtualProtect y strcmp suelen usarse para modificar la memoria de un programa, lo que es un truco que usan los virus para esconderse 
        * GetProcAddress función común en virus que intentan encontrar funciones importantes para infectar o manipular otros procesos
        * CreateRemoteThread a veces la usan para meterse en otros procesos que están funcionando
    
    * sí, hay indicios sospechosos de que este programa podría estar haciendo algo malicioso, basándonos en las funciones que usa y su relación con lo que vimos en la Tabla 3 del artículo

In [20]:
timestamp = pe.FILE_HEADER.TimeDateStamp
compilation_date = datetime.fromtimestamp(timestamp, timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

print(f"🗓️ El ejecutable fue compilado el: {compilation_date}")

# Cerrar el archivo PE
pe.close()

🗓️ El ejecutable fue compilado el: 2010-11-20 09:05:05


In [24]:
executables = [
    "examples/sample_vg655_25th.exe",
    "examples/sample_qwrty_dk2"
]

for exe_file in executables:
    print(f"\n🔍 Análisis de ensamblador para: {exe_file}")

    # Cargar el ejecutable
    pe = pefile.PE(exe_file)

    # Obtener punto de entrada
    entrypoint = pe.OPTIONAL_HEADER.AddressOfEntryPoint
    entrypoint_address = entrypoint + pe.OPTIONAL_HEADER.ImageBase

    # Obtener los primeros 200 bytes desde el punto de entrada
    binary_code = pe.get_memory_mapped_image()[entrypoint:entrypoint+200]

    # Inicializar el desensamblador
    disassembler = Cs(CS_ARCH_X86, CS_MODE_32)

    # Desensamblar y mostrar las instrucciones
    for instruction in disassembler.disasm(binary_code, entrypoint_address):
        print(f"0x{instruction.address:x}:\t{instruction.mnemonic}\t{instruction.op_str}")

    # Cerrar el archivo
    pe.close()


🔍 Análisis de ensamblador para: examples/sample_vg655_25th.exe
0x4077ba:	push	ebp
0x4077bb:	mov	ebp, esp
0x4077bd:	push	-1
0x4077bf:	push	0x40d488
0x4077c4:	push	0x4076f4
0x4077c9:	mov	eax, dword ptr fs:[0]
0x4077cf:	push	eax
0x4077d0:	mov	dword ptr fs:[0], esp
0x4077d7:	sub	esp, 0x68
0x4077da:	push	ebx
0x4077db:	push	esi
0x4077dc:	push	edi
0x4077dd:	mov	dword ptr [ebp - 0x18], esp
0x4077e0:	xor	ebx, ebx
0x4077e2:	mov	dword ptr [ebp - 4], ebx
0x4077e5:	push	2
0x4077e7:	call	dword ptr [0x4081c4]
0x4077ed:	pop	ecx
0x4077ee:	or	dword ptr [0x40f94c], 0xffffffff
0x4077f5:	or	dword ptr [0x40f950], 0xffffffff
0x4077fc:	call	dword ptr [0x4081c0]
0x407802:	mov	ecx, dword ptr [0x40f948]
0x407808:	mov	dword ptr [eax], ecx
0x40780a:	call	dword ptr [0x4081bc]
0x407810:	mov	ecx, dword ptr [0x40f944]
0x407816:	mov	dword ptr [eax], ecx
0x407818:	mov	eax, dword ptr [0x4081b8]
0x40781d:	mov	eax, dword ptr [eax]
0x40781f:	mov	dword ptr [0x40f954], eax
0x407824:	call	0x40793f
0x407829:	cmp	dword ptr [0x4